## 最小二乘法 求解r a b d参数

In [3]:
import numpy as np
from scipy.optimize import minimize

# Observational data
t_obs = np.array([0, 1, 2, 3, 4, 5, 6])
x_obs = np.array([1000, 2996, 217, 29, 22, 49, 214])
y_obs = np.array([500, 1737, 3069, 2017, 1266, 800, 537])

In [1]:
# ODE system
def predator_prey(t, u, r, a, b, d):
    x, y = u
    dxdt = r * x - a * x * y
    dydt = -d * y + b * x * y
    return np.array([dxdt, dydt])

# Adams predictor-corrector method
def adams_pc(f, t0, u0, t_end, h, params):
    r, a, b, d = params
    t_values = np.arange(t0, t_end + h, h)
    n = len(t_values)
    u_values = np.zeros((n, len(u0)))
    u_values[0] = u0
    
    for i in range(1, min(4, n)):
        t = t_values[i-1]
        u = u_values[i-1]
        k1 = f(t, u, r, a, b, d)
        k2 = f(t + h/2, u + h/2 * k1, r, a, b, d)
        k3 = f(t + h/2, u + h/2 * k2, r, a, b, d)
        k4 = f(t + h, u + h * k3, r, a, b, d)
        u_values[i] = u + h/6 * (k1 + 2*k2 + 2*k3 + k4)
    
    for i in range(4, n):
        t = t_values[i-1]
        f_n = f(t_values[i-1], u_values[i-1], r, a, b, d)
        f_n1 = f(t_values[i-2], u_values[i-2], r, a, b, d)
        f_n2 = f(t_values[i-3], u_values[i-3], r, a, b, d)
        f_n3 = f(t_values[i-4], u_values[i-4], r, a, b, d)
        u_pred = u_values[i-1] + h/24 * (55 * f_n - 59 * f_n1 + 37 * f_n2 - 9 * f_n3)
        f_pred = f(t_values[i], u_pred, r, a, b, d)
        u_values[i] = u_values[i-1] + h/24 * (9 * f_pred + 19 * f_n - 5 * f_n1 + f_n2)
    
    return t_values, u_values

In [2]:
# Solve ODE at observation points
def solve_ode_at_obs(params):
    t0 = 0
    u0 = np.array([1000, 500])
    t_end = 6
    h = 0.01
    t_values, u_values = adams_pc(predator_prey, t0, u0, t_end, h, params)
    indices = (t_obs / h).astype(int)
    return u_values[indices, 0], u_values[indices, 1]

# Error function
def error_function(params):
    x_pred, y_pred = solve_ode_at_obs(params)
    error = np.sum((x_pred - x_obs)**2 + (y_pred - y_obs)**2)
    return error

In [5]:
# Initial guess and optimization
initial_guess = [3, 0.002, 0.0006, 0.5]
result = minimize(error_function, initial_guess, method='Nelder-Mead')

# Results
best_params = result.x
print("最佳参数 r, a, b, d:", best_params)
print("最佳参数 r, a, b, d: [{:.2f}, {:.4f}, {:.5f}, {:.2f}]".format(*best_params))
print("最小误差:", result.fun)

最佳参数 r, a, b, d: [2.67765834e+00 1.84535534e-03 7.92913181e-04 4.83842516e-01]
最佳参数 r, a, b, d: [2.68, 0.0018, 0.00079, 0.48]
最小误差: 0.5067370795180575
